Helen Nguyen

Backtest a strategy that buys when a stock's 50-day moving average crosses above its 200-day moving average (a "Golden Cross" signal) and sells when the 50-day moving average crosses below the 200-day moving average (a "Death Cross" signal).

In [1]:
import pandas as pd
import numpy as np
import datetime
# !pip install yfinance
import yfinance as yf
import matplotlib.pyplot as plt

In [2]:
stock = str(input("Input stock ticker: "))

Input stock ticker: RACE


In [3]:
# Define a range from when until when we want the data
start_year = int(input("Input start year: "))
end_year = int(input("Input end year: "))

start_date = datetime.datetime(start_year, 1, 1) # the format is YYYY-MM-DD
end_date = datetime.datetime(end_year, 12, 31)

Input start year: 2018
Input end year: 2022


In [4]:
# Load stock data
df = yf.download(stock, start=start_date, end=end_date)

[*********************100%***********************]  1 of 1 completed


In [5]:
df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,104.120003,105.379997,103.650002,105.150002,101.562737,546800
2018-01-03,106.339996,107.389999,106.339996,107.169998,103.513817,426500
2018-01-04,112.610001,112.639999,111.019997,111.309998,107.512581,882800
2018-01-05,112.739998,113.019997,112.029999,112.129997,108.304604,408100
2018-01-08,112.820000,113.610001,112.580002,113.519997,109.647171,451200


In [6]:
# Calculate the 50-day and 200-day moving averages
df["50d"] = df["Close"].rolling(window=50).mean()
df["200d"] = df["Close"].rolling(window=200).mean()

In [7]:
# Create a column to store the buy/sell signals
df["Signal"] = None

In [8]:
# Generate buy signals
df.loc[(df["50d"] > df["200d"]) & (df["50d"].shift(1) <= df["200d"].shift(1)), "Signal"] = 1

In [9]:
# Generate sell signals
df.loc[(df["50d"] < df["200d"]) & (df["50d"].shift(1) >= df["200d"].shift(1)), "Signal"] = -1

In [10]:
df["Signal"].fillna(value=0, inplace=True)

In [11]:
df[(df["Signal"] == 1) | (df["Signal"] == -1)]

,Open,High,Low,Close,Adj Close,Volume,50d,200d,Signal
Date,,,,,,,,,
2018-10-23,112.970001,116.050003,112.330002,115.279999,112.144249,1541800,128.178599,128.267950,-1
2019-03-22,131.050003,131.339996,128.500000,128.649994,125.150558,411900,123.699800,123.513250,1
2020-03-25,149.339996,156.940002,147.979996,154.059998,151.123611,1378400,160.634399,160.803450,-1
2020-06-16,173.289993,173.289993,168.589996,169.690002,167.733414,242100,160.534199,160.249500,1
2022-03-15,193.449997,195.300003,192.509003,195.199997,193.899551,275800,225.659600,226.823449,-1
2022-12-07,220.990005,222.720001,219.770004,220.080002,220.080002,194800,201.425800,200.905800,1


In [12]:
df['Daily P&L'] = df['Adj Close'].diff() * df['Signal']
df['Total P&L'] = df['Daily P&L'].cumsum()

In [13]:
df.tail()

,Open,High,Low,Close,Adj Close,Volume,50d,200d,Signal,Daily P&L,Total P&L
Date,,,,,,,,,,,
2022-12-23,210.800003,212.309998,209.880005,211.009995,211.009995,222000,209.000000,201.83965,0,-0.0,-14.886627
2022-12-27,211.800003,212.970001,210.300003,212.000000,212.000000,190300,209.627400,201.94740,0,0.0,-14.886627
2022-12-28,212.000000,213.240005,210.035004,210.250000,210.250000,163200,210.060600,202.03400,0,-0.0,-14.886627
2022-12-29,213.789993,216.929001,213.699997,216.589996,216.589996,189300,210.572000,202.14095,0,0.0,-14.886627
2022-12-30,215.539993,216.289993,212.804993,214.220001,214.220001,141200,211.104999,202.16755,0,-0.0,-14.886627


&rarr; This strategy leads to a cumulative loss of $14.89 per share over the five-year period from Jan 1 2019 to Dec 31 2022 for RACE.
